In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

from generate_inputs import RUNS, A0_VALUES, DEFAULT_TIME, DEFAULT_DT, M_VALUES, DEFAULT_M
from utils import compute_mean_square_error, FONT, COLORS, Q_ERROR_POINTS

time_points = [i * DEFAULT_DT for i in range(int(DEFAULT_TIME / DEFAULT_DT))]
os.makedirs('../analysis', exist_ok=True)

In [ ]:
print(f"A0 values: {A0_VALUES}")
print(f"M values: {M_VALUES}")

In [ ]:
def calculate_q(exits_data, steady_state, min_q=0.0, max_q=2.0):
    filtered_data = exits_data[exits_data.index > steady_state]

    x_values = filtered_data.index
    y_values = filtered_data

    x0 = x_values[0]
    y0 = y_values.iloc[0]

    fit_x_values = x_values - x0
    fit_y_values = y_values - y0

    # Define the mean square error function for a given q
    def mse_for_q(q):
        return compute_mean_square_error(q, fit_x_values, fit_y_values)

    # Use minimize_scalar to find the best q
    result = minimize_scalar(mse_for_q, bounds=(min_q, max_q), method='bounded')

    best_q = result.x
    min_error = result.fun

    return x0, y0, best_q, min_error


def calculate_q_with_errors(exits_data, steady_state, min_q=0.0, max_q=2.0, tries=Q_ERROR_POINTS):
    filtered_data = exits_data[exits_data.index > steady_state]

    x_values = filtered_data.index
    y_values = filtered_data

    x0 = x_values[0]
    y0 = y_values.iloc[0]

    fit_x_values = x_values - x0
    fit_y_values = y_values - y0

    # Define the mean square error function for a given q
    def mse_for_q(q):
        return compute_mean_square_error(q, fit_x_values, fit_y_values)

    # Use minimize_scalar to find the best q
    result = minimize_scalar(mse_for_q, bounds=(min_q, max_q), method='bounded')
    best_q = result.x
    min_error = result.fun

    # Calculate errors for plotting
    q_values = np.linspace(min_q, max_q, tries)
    q_errors = [mse_for_q(q) for q in q_values]

    # Insert precise best_q and min_error at the correct position
    insert_idx = np.searchsorted(q_values, best_q)
    q_values = np.insert(q_values, insert_idx, best_q)
    q_errors = np.insert(q_errors, insert_idx, min_error)

    return x0, y0, best_q, min_error, q_values, q_errors

# Ejercicio A

# Ejercicio B

In [ ]:

b_q_means = np.zeros(len(A0_VALUES))
b_q_std = np.zeros(len(A0_VALUES))

m_index = M_VALUES.index(DEFAULT_M)

for i in range(len(A0_VALUES)):

    b_q_values = np.zeros(RUNS)
    b_errors = np.zeros(RUNS)

    print(time.time(), i)

    for j in range(RUNS):
        print(j, end='\r')
        exits_file = f"../outputs/3_{m_index}_{i}_{j}_exits.csv"
        exits_data = pd.read_csv(exits_file)
        # print(exits_data)
        exits_data["time"] = (exits_data["time"] / DEFAULT_DT).round() * DEFAULT_DT

        crossings_count = exits_data["time"].value_counts().reindex(time_points, fill_value=0).cumsum()

        _, _, q, error = calculate_q(crossings_count, 200.0)

        b_q_values[j] = q
        b_errors[j] = error

    b_q_means[i] = np.mean(b_q_values)
    b_q_std[i] = np.std(b_q_values)

In [ ]:
print(b_q_means)
print(b_q_std)

In [ ]:
b_q_means_fit = [b_q_mean - b_q_means[0] for b_q_mean in b_q_means]
b_a0_fit = [a0 - A0_VALUES[0] for a0 in A0_VALUES]

print(b_q_means_fit)
print(b_a0_fit)

b_possible_vals = np.linspace(0.0, 1.0, Q_ERROR_POINTS)
errors = [compute_mean_square_error(b, b_a0_fit, b_q_means_fit) for b in b_possible_vals]

b_min_b_error_idx = np.argmin(errors)
b_min_b_error = errors[b_min_b_error_idx]
b_best_b = b_possible_vals[b_min_b_error_idx]

print(f"Best b: {b_best_b}")
print(f"Min error: {b_min_b_error}")

In [ ]:
# Graph q_means vs a0 with error bars
# plt.scatter(A0_VALUES, b_q_means)
fig, ax = plt.subplots(1, 2, figsize=(10, 6), width_ratios=[9, 1])

ax[0].errorbar(A0_VALUES, b_q_means, yerr=b_q_std, fmt='o--', capsize=5, color=COLORS[0])
ax[0].plot(A0_VALUES, [b_best_b * (a0 - A0_VALUES[0]) + b_q_means[0] for a0 in A0_VALUES],
         label=f"$Q = {b_best_b:.3f} \\, \\mathrm{{s/cm}} \\, (a_0 - {A0_VALUES[0]} \\, \\mathrm{{cm/s^2}}) + {b_q_means[0]:.3f} \\, \\mathrm{{s^{{-1}}}}$",
         color=COLORS[1])

# Q = m_p * a0 / R
# b = m_p / R
# R = m_p / b
resistance = 1.0 / b_best_b
print(f"R: {resistance}")

ax[1].axis('off')
ax[1].text(
    0.1, 0.5,  # Adjust x and y values as needed for positioning
    f"$R = {resistance:.3f} \\; \\mathrm{{cm/s}}$",
    fontsize=12,
    verticalalignment='center',
    horizontalalignment='left',
)

ax[0].set_xticks(A0_VALUES)
ax[0].set_xticklabels([f"{val}" for val in A0_VALUES], fontsize=14)
ax[0].tick_params(axis='y', labelsize=14)
ax[0].set_xlabel("$A_0 \\; (\\mathrm{cm/s^2})$", fontdict=FONT)
ax[0].set_ylabel("$Q \\; (\\mathrm{s^{-1}})$", fontdict=FONT)
ax[0].legend(loc='upper left')

plt.tight_layout()

plt.savefig("../analysis/q_vs_a0.png", dpi=300)
plt.show()
plt.clf()

plt.figure(figsize=(10, 6))

plt.plot(b_possible_vals, errors, marker='.', color='blue')
min_idx = np.argmin(errors)
plt.plot(b_possible_vals[min_idx], errors[min_idx], 'o', color='blue')
plt.xlabel("$m_p / R \\; (\\mathrm{s/cm})$", fontdict=FONT)
plt.ylabel("ECM", fontdict=FONT)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim(0.0, 0.5)
plt.ylim(0, 0.2)
plt.tight_layout()
plt.savefig("../analysis/q_vs_a0_error.png", dpi=300)
plt.show()
plt.clf()

# Ejercicio C

In [ ]:
c_q_means = np.zeros((len(M_VALUES), len(A0_VALUES)))
c_q_std = np.zeros((len(M_VALUES), len(A0_VALUES)))

for i in range(len(M_VALUES)):
    for j in range(len(A0_VALUES)):
        c_q_values = np.zeros(RUNS)
        c_errors = np.zeros(RUNS)

        for k in range(RUNS):
            print(j, end='\r')
            exits_file = f"../outputs/3_{i}_{j}_{k}_exits.csv"
            exits_data = pd.read_csv(exits_file)
            # print(exits_data)
            exits_data["time"] = (exits_data["time"] / DEFAULT_DT).round() * DEFAULT_DT

            crossings_count = exits_data["time"].value_counts().reindex(time_points, fill_value=0).cumsum()

            _, _, q, error = calculate_q(crossings_count, 200.0)

            c_q_values[k] = q
            c_errors[k] = error

        c_q_means[i][j] = np.mean(c_q_values)
        c_q_std[i][j] = np.std(c_q_values)


In [ ]:
print(c_q_means)
print(c_q_std)

In [ ]:
c_inv_resistance = np.zeros(len(M_VALUES))
c_inv_resistance_ecm = np.zeros(len(M_VALUES))

c_inv_resistance_errors = []
c_inv_resistance_values = []

for i in range(len(M_VALUES)):
    c_q_means_fit = [c_q_mean - c_q_means[i][0] for c_q_mean in c_q_means[i]]
    c_a0_fit = [a0 - A0_VALUES[0] for a0 in A0_VALUES]


    def mse_for_c(c):
        return compute_mean_square_error(c, c_a0_fit, c_q_means_fit)


    result = minimize_scalar(mse_for_c, bounds=(0.0, 6.0), method='bounded')

    c_inv_resistance[i] = result.x
    c_inv_resistance_ecm[i] = result.fun

    c_inv_res_v = np.linspace(result.x - 0.2, result.x + 0.2, Q_ERROR_POINTS)
    c_inv_res_e = [mse_for_c(c) for c in c_inv_res_v]

    c_inv_min_idx = np.searchsorted(c_inv_res_v, result.x)
    c_inv_res_v = np.insert(c_inv_res_v, c_inv_min_idx, result.x)
    c_inv_res_e = np.insert(c_inv_res_e, c_inv_min_idx, result.fun)

    c_inv_resistance_errors.append(c_inv_res_e)
    c_inv_resistance_values.append(c_inv_res_v)


In [ ]:
print(c_inv_resistance)
print(c_inv_resistance_ecm)

In [ ]:
plt.figure(figsize=(10, 6))
for i in range(len(M_VALUES)):
    plt.errorbar(A0_VALUES, c_q_means[i], yerr=c_q_std[i], fmt='--o', capsize=5, label=f"{M_VALUES[i]} obstáculos",
                 color=COLORS[i])
    # plt.plot(A0_VALUES, [c_inv_resistance[i] * (a0 - A0_VALUES[0]) + c_q_means[i][0] for a0 in A0_VALUES],
    #          label=f"y = {c_inv_resistance[i]:.3f}x (ECM: {formatter(c_inv_resistance_ecm[i],None)})",
    #          color=COLORS[i],)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), framealpha=1.0, ncol=3)

# plt.subplots_adjust(top=0.95)
plt.xlabel("$A_0 \\; (\\mathrm{cm/s^2})$", fontdict=FONT)
plt.ylabel("$Q \\; (\\mathrm{s^{-1}})$", fontdict=FONT)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()

plt.savefig("../analysis/q_vs_a0_m.png", dpi=300)
plt.show()
plt.clf()


In [ ]:

plt.figure(figsize=(10, 6))
for i in range(len(M_VALUES)):
    plt.errorbar(A0_VALUES, c_q_means[i], yerr=c_q_std[i], fmt='--o', capsize=5, alpha=0.25, color=COLORS[i])
    plt.plot(A0_VALUES, [c_inv_resistance[i] * (a0 - A0_VALUES[0]) + c_q_means[i][0] for a0 in A0_VALUES],
             label=f"$Q = {c_inv_resistance[i]:.3f} \\, \\mathrm{{s/cm}} \\, (a_0 - {A0_VALUES[0]} \\,\\mathrm{{cm/s^2}}) + {c_q_means[i][0]:.3f} \\, \\mathrm{{s^{{-1}}}}$",
             color=COLORS[i], )

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), framealpha=1.0, ncol=2)

# plt.subplots_adjust(top=0.95)
plt.xlabel("$A_0 \\; (\\mathrm{cm/s^2})$", fontdict=FONT)
plt.ylabel("$Q \\; (\\mathrm{s^{-1}})$", fontdict=FONT)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()

plt.savefig("../analysis/q_vs_a0_m_lines.png", dpi=300)
plt.show()
plt.clf()

plt.figure(figsize=(10, 6))
for i in range(len(M_VALUES)):
    plt.plot(c_inv_resistance_values[i], c_inv_resistance_errors[i], marker='.', label=f"{M_VALUES[i]} obstáculos",
             color=COLORS[i])
    min_idx = np.argmin(c_inv_resistance_errors[i])
    plt.plot(c_inv_resistance_values[i][min_idx], c_inv_resistance_errors[i][min_idx], 'o', color=COLORS[i])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), framealpha=1.0, ncol=3)
plt.xlabel("$m/R \\; (\\mathrm{s/cm})$", fontdict=FONT)
plt.ylabel("ECM", fontdict=FONT)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()

plt.savefig("../analysis/q_vs_a0_m_error.png", dpi=300)
plt.show()
plt.clf()



In [ ]:
c_resistance = 1 / c_inv_resistance
c_resistance_error = 1 / (c_resistance ** 2) * c_inv_resistance_ecm
print(c_resistance)
print(c_resistance_error)

In [ ]:
# plt.errorbar(M_VALUES, c_resistance, yerr=c_resistance_error, fmt="o--", capsize=5 , color='blue')
plt.figure(figsize=(10, 6))
plt.plot(M_VALUES, c_resistance, "o--", color='blue')
plt.xlabel("Obstáculos", fontdict=FONT)
plt.ylabel("$R \\; (\\mathrm{g \\cdot cm/s})$", fontdict=FONT)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
# plt.ylim(0, 6.5)
plt.tight_layout()

plt.savefig("../analysis/resistance_vs_m.png", dpi=300)
plt.show()
plt.clf()